In [1]:
import csv
import pathlib
from IPython.display import display, HTML
import ipywidgets
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

csv_path = pathlib.Path('./employees-pension-contribution.csv')
with open(csv_path, encoding='utf-8', newline='') as f:
    EMPLOYEES_PENSION_GRADES = list(csv.DictReader(f))


In [2]:
def rate_by_delay(year_receipt_begins):
    """:returns: 年金受給開始年による変動率"""
    # https://www.nenkin.go.jp/service/jukyu/roureinenkin/kuriage-kurisage/20140421-01.html
    monthly_decrease_rate = -0.004

    # https://www.nenkin.go.jp/service/jukyu/roureinenkin/kuriage-kurisage/20140421-02.html
    monthlyIncreaseRate= 0.007

    baseYear = 65
    yearDelay = year_receipt_begins - baseYear
    if (yearDelay < 0):
        return 1 + abs(yearDelay) * monthly_decrease_rate * 12
    else:
        return 1 + yearDelay * monthlyIncreaseRate * 12

In [3]:
def geo_series(
    first_term,
    common_ratio,
    number_of_terms,
):
    """:returns: Gemeometric series"""
    if common_ratio == 1:
        return first_term * number_of_terms
    elif common_ratio > 1:
        return first_term * (1 - common_ratio ** (number_of_terms)) / (1 - common_ratio)
    else:
        raise Exception('Invalid common ratio')


In [4]:
def no_pension(
    apy,
    monthly_payment,
    year_investment_ends,
):
    """年金なし"""
    years = range(1, year_investment_ends + 1)
    yearly_rate = 1 + apy
    monthyly_rate = yearly_rate ** (1 / 12)
    investment_sum_per_year = geo_series(monthly_payment, monthyly_rate, 12)
    return list(map(lambda year: geo_series(investment_sum_per_year, yearly_rate, year), years))


In [5]:
def national_pension(
    apy,
    monthly_payment,
    year_investment_ends,
    year_receipt_begins,
    option,
):
    """国民年金（老齢基礎年金）"""
    monthly_contribution, monthly_benefit = monthly_national_cb(option, year_receipt_begins)
    monthly_surplus = monthly_payment - monthly_contribution
    if monthly_surplus < 0:
        raise Exception("monthly surplus is less than zero")

    year_contribution_ends = min(year_investment_ends, 40)

    yearly_rate = 1 + apy
    monthly_rate = yearly_rate ** (1 / 12)
    surplus_sum_per_year = geo_series(monthly_surplus, monthly_rate, 12)
    pension_sum_per_year = geo_series(monthly_benefit, monthly_rate, 12)
    investment_sum_per_year = geo_series(monthly_payment, monthly_rate, 12)
    balances = [0] * year_investment_ends
    balances[0] = surplus_sum_per_year
    for i in range(1, year_contribution_ends):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate  + surplus_sum_per_year
    for i in range(40, year_receipt_begins - 20 - 1):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate + investment_sum_per_year
    for i in range(year_receipt_begins - 20 - 1, year_investment_ends):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate + investment_sum_per_year + pension_sum_per_year
    return balances

def monthly_national_cb(option, year_receipt_begins) -> tuple[int, int]:
    """National Pension's contribution and benefit for a month
    :returns: a tuple of contribution and benefit of national pension for a month
    """
    options = ['standard', 'additive', 'exemptFull', 'exempt34', 'exempt12', 'exempt14']
    if not(option in options):
        raise Exception(f'Unexpected option value {option} in monthly_national_cb()')
    
    standard_contribution = 16610
    if option == 'standard':
        monthly_contribution = standard_contribution
        monthly_benefit = 780900 * rate_by_delay(year_receipt_begins) / 12
    elif option == 'additive':
        monthly_contribution = standard_contribution + 400
        monthly_benefit = (780900 + 96000) * rate_by_delay(year_receipt_begins) / 12
    elif option == 'exemptFull':
        benefit_rate = 1 / 2
        monthly_contribution = 0
        monthly_benefit = 780900 * rate_by_delay(year_receipt_begins) / 12 * benefit_rate
    elif option == 'exempt34':
        benefit_rate = 5 / 8
        monthly_contribution = standard_contribution * 1 / 4
        monthly_benefit = 780900 * rate_by_delay(year_receipt_begins) / 12 * benefit_rate
    elif option == 'exempt12':
        benefit_rate = 6 / 8
        monthly_contribution = standard_contribution * 1 / 2
        monthly_benefit = 780900 * rate_by_delay(year_receipt_begins) / 12 * benefit_rate
    elif option == 'exempt14':
        benefit_rate = 7 / 8
        monthly_contribution = standard_contribution * 3 / 4
        monthly_benefit = 780900 * rate_by_delay(year_receipt_begins) / 12 * benefit_rate
    else:
        raise Exception(f'Unexpected option value {option} in monthly_national_cb()')
    return (monthly_contribution, monthly_benefit)


In [6]:
def employees_pension(
    apy,
    monthly_payment,
    year_investment_ends,
    year_receipt_begins,
    grade,
    include_national,
):
    """厚生年金（老齢厚生年金）"""
    grade_info = EMPLOYEES_PENSION_GRADES[grade - 1]
    standard_income = grade_info['standard_income']
    contribution_str = grade_info['employee_contribution']
    monthly_contribution = int(float(contribution_str))
    monthly_benefit = int(standard_income) * 5.481 / 1000 * 480 * rate_by_delay(year_receipt_begins) / 12 + (780900  * rate_by_delay(year_receipt_begins) / 12 if include_national else 0)
    monthly_surplus = monthly_payment - monthly_contribution
    if monthly_surplus < 0:
        return Exception(f'employees pension grade {grade} not calculated: monthly_payment {monthly_payment} is less than monthly_contribution {monthly_contribution}.')


    year_contribution_ends = min(year_investment_ends, 40)

    yearly_rate = 1 + apy
    monthly_rate = yearly_rate ** (1 / 12)
    surplus_sum_per_year = geo_series(monthly_surplus, monthly_rate, 12)
    pension_sum_per_year = geo_series(monthly_benefit, monthly_rate, 12)
    investment_sum_per_year = geo_series(monthly_payment, monthly_rate, 12)
    balances = [0] * year_investment_ends

    balances[0] = surplus_sum_per_year
    for i in range(1, year_contribution_ends):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate + surplus_sum_per_year
    for i in range(40, year_receipt_begins - 20 - 1):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate + investment_sum_per_year
    for i in range(year_receipt_begins - 20 - 1, year_investment_ends):
        last_balance = balances[i - 1]
        balances[i] = last_balance * yearly_rate + investment_sum_per_year + pension_sum_per_year

    return balances


In [7]:
def plot(
    datum,
    min_year,
    max_year,
    log_scale,
    legend
):
    
    def sub_cmap(cmap, vmin, vmax):
        return lambda v: cmap(vmin + (vmax - vmin) * v)

    # # Dark background
#     plt.style.use('dark_background')
    # # White background
    mpl.style.use('default')

    mpl.rc('font', family='Hiragino Maru Gothic Pro')
    plt.rcParams.update({'font.size': 10})
    
    fig, ax = plt.subplots()
    fig.set_size_inches(25, 10)
    
    years = list(range(min_year, max_year + 1))


    no_pension_data, no_pension_label = datum.pop()
    national_option_n = 6
    national_datum = datum[:6]
    employees_datum = datum[6:]
    
    # red, blue
    emp_cmap = plt.cm.get_cmap('PuBu')
    nat_cmap = plt.cm.get_cmap('YlOrRd')

    # Plot no pension
    ax.plot(years, no_pension_data[min_year - 1:max_year], label=no_pension_label, c='#000', linestyle=(0, (5, 5)), linewidth=1.5)

    # Plot employees pension
    emp_cmap = sub_cmap(plt.cm.get_cmap('PuBu'), 0.1, 0.9)
    for i, (data, label) in enumerate(employees_datum):
        color = emp_cmap(i / (len(employees_datum) - 1))
        ax.plot(years, data[min_year - 1:max_year], label=label, c=color)
        
    # Plot national pension
    nat_cmap = sub_cmap(plt.cm.get_cmap('YlOrRd'), 0.2, 0.8)
    for i, (data, label) in enumerate(national_datum):
        color = nat_cmap(i / (len(national_datum) - 1))
        ax.plot(years, data[min_year - 1:max_year], label=label, c=color, linestyle='dotted', linewidth=2)

    # Log scale
    if log_scale:
        ax.set_yscale('log')

    if legend:
        plt.legend(loc="upper left")
        
    ax.set_xlabel('(歳)', loc='right')
    ax.set_ylabel('残高 (万円)', loc='top')
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, pos: int(x + 20)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, pos: int(y / 10000)))
    plt.show()


In [8]:
def feed_plot(
    apy,
    year_receipt_begins,
    monthly_payment,
    min_year,
    max_year,
    log_scale,
    legend
):
    min_year = min_year - 20
    max_year = max_year - 20

    apy = apy / 100
    
    national_options = {
        'exemptFull': '国民年金 (全額免除)',
        'exempt34': '国民年金 (3/4免除)',
        'exempt12': '国民年金 (1/2免除)',
        'exempt14': '国民年金 (1/4免除)',
        'standard': '国民年金(オプションなし)',
        'additive': '国民年金 (付加年金）',
    }
        
    datum = []
    for option, name in national_options.items():
        datum.append((national_pension(apy, monthly_payment, max_year, year_receipt_begins, option), name))

    for grade in range(1, 33):
        datum.append((employees_pension(apy, monthly_payment, max_year, year_receipt_begins, grade, True), f'厚生年金 (等級{grade}）'))
        
    datum.append((no_pension(apy, monthly_payment, max_year), '年金なし'))


    ok_datum, exception_datum = [], []
    for data in datum:
        if isinstance(data[0], Exception):
            exception_datum.append(data)
        else:
            ok_datum.append(data)
        
    plot(
        ok_datum,
        min_year,
        max_year,
        log_scale,
        legend,
    )

    for data in exception_datum:
        print(f'❗️ {data[0]}')

In [9]:
display(HTML('<style> .widget-label { width:8rem !important;} .widget-dropdown, .widget-hslider {width: 50%;} </style>'))

max_year_limit = 200
layout = ipywidgets.Layout(width='100%')
apy = ipywidgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='年利回り(%)')
year_receipt_begins = ipywidgets.IntSlider(value=65, min=60, max=75, step=1, description='受給開始年')
min_year = ipywidgets.IntSlider(value=40, min=1+20, max=max_year_limit+20, step=1, description='表示する最初の年')
max_year = ipywidgets.IntSlider(value=100, min=1+20, max=max_year_limit+20, step=1, description='表示する最後の年')
# contribution cost for national pension with additional pension
addtional_min_cost = 17010
# contribution cost for employees pension grade 32
employees_min_cost = 59475
monthly_payment = ipywidgets.IntSlider(value=employees_min_cost, min=addtional_min_cost, max=200000, step=5000, description='月々の積立金額')
log_scale = ipywidgets.Checkbox(value=False, description='対数スケール')
sequential = ipywidgets.Checkbox(value=True, description='連続的な色調')
legend = ipywidgets.Checkbox(value=True, description='ラベル')
w = ipywidgets.interactive(
    feed_plot,
    apy=apy,
    year_receipt_begins=year_receipt_begins,
    monthly_payment=monthly_payment,
    min_year=min_year,
    max_year=max_year,
    log_scale=log_scale,
    legend=legend,
)
display(ipywidgets.HBox([w]))